# Imports 

In [23]:
import yt_dlp
from bs4 import BeautifulSoup
import urllib.request
import os
import numpy as np
import cv2
from time import time as timer
import sys
import pandas as pd

## Get URLS

In [14]:
def SearchVid(search):
    responce = urllib.request.urlopen('https://www.youtube.com/results?search_query='+search)

    soup = BeautifulSoup(responce)    
    divs = soup.find_all("div", { "class" : "yt-lockup-content"})


    for i in divs:
        href= i.find('a', href=True)
        print(href.text,  "\nhttps://www.youtube.com"+href['href'], '\n')
        with open(SearchString.replace("%20", "_")+'.txt', 'a') as writer:
            writer.write("https://www.youtube.com"+href['href']+'\n')

print("What are you looking for?")
SearchString = input()
SearchVid(SearchString.replace(" ", "%20"))

What are you looking for?
wrong-way drivers


In [ ]:
ydl_opts = {}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/results?search_query=drifting++dashcam&sp=EgIYAQ%253D%253D&'])

## Rename Files 

In [12]:
from os import listdir
from os.path import isfile, join

In [15]:
def rename(name,path):
    mypath= path
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    i=0
    for f in onlyfiles:
        old_file = os.path.join(mypath, str(f))
        new_file = os.path.join(mypath,name+f'_{i}.mp4' )
        os.rename(old_file, new_file)
        i+=1
        

In [21]:
rename('wr_way','/Users/workplace/Documents/GitHub/data/wr_way')

## Extract Frames 

In [29]:
# setup paths
FPS_OUTPUT = 2
pwd = '/Users/workplace/Documents/GitHub/data/'
path_videos = pwd + 'wr_way/'
path_data = pwd + 'data/'

In [ ]:
# read video paths
paths = os.listdir(path_videos)
paths = [path_videos + v for v in paths if v != '.DS_Store']
paths

In [ ]:
# create data folder
if not os.path.exists(path_data):
    os.makedirs(path_data)

for c, path in enumerate(paths):

    # extract video filename from path
    filename = path.split("/")[-1].split(".")[0]
    print(filename)

    if not os.path.exists(path_data + filename):

        print("Extracting frames from video {}/{}: {}".format(c+1,len(paths),filename))

        # create directory for this video's frames
        os.makedirs(path_data + filename)

        # open video
        vidcap = cv2.VideoCapture(path)

        # get fps
        print("video FPS {}".format(vidcap.get(cv2.CAP_PROP_FPS)))
        fps = vidcap.get(cv2.CAP_PROP_FPS)
        fps_savecheck = fps/FPS_OUTPUT

        # read frames and save to images at fps_save
        success,image = vidcap.read()
        count_frame = 0
        count_saved = 0
        success = True

        fps_savecheck_progress = 0

        # only extract a subset of frames
        while success and count_saved<=100:
        # extract all frames
            # save frame at desired framerate, indexing filename from 0
            if count_frame > fps_savecheck_progress:
                cv2.imwrite(path_data + filename + '/' + filename + "_%06d.jpg" % count_saved, image)
                fps_savecheck_progress += fps_savecheck
                count_saved += 1
            success,image = vidcap.read()
            count_frame += 1

        # close video file
        vidcap.release()
        
    else:
        print("Frames already extracted from video {}/{}".format(c+1,len(paths)))

# Done
print("Done extracting frames from {} videos".format(len(paths)))
